<a href="https://colab.research.google.com/github/svvsaga/datascience_workshop/blob/main/workshop_sesjon2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Workshop sesjon 2: Analyse med BigQuery


Google's BigQuery er en serverless datawarehus løsning som støtter analyse av store datamengder uten å kreve at brukeren sette opp infrastruktur i bakgrunnen.
BigQuery bruker [standard SQL](https://www.w3schools.com/sql/sql_intro.asp) som språk. I tilleg finnes det noen ekstra funksjoner som gjør det enklere å kjøre komplekse analyser, til og med noen støttede GIS funksjonalitet.

I denne sesjonen skal vi gå gjennom noen av de fordelene man oppnå ved bruk av BigQuery og vise noen triks for å komme i gang med det.

Koden vi går gjennom kan både kjøres i denne noteboken og  i BigQuery UI gjennom nettleseren. **Vi anbefaler å utforske/teste spørringer i BigQuery UI** der man få noen fordeler  som f.eks. en live query validator samt estimert bruk av datavolumen og kan utforske datastrukturen mer interaktivt.


## Autentisering med colab notebook
Hvis notebook brukes til å kjøre spørringer, er det nødvendig å autentisere seg (kjør cellen under).

In [1]:
from google.colab import auth
auth.authenticate_user()

In [2]:
project = "saga-analytics-temp" # set project-name globally

## Grunnlegende spørringer
BigQuery bruker standard SQL som språk. Cellen under viser en enkelt spørring som bare returnerer alle rader fra vår trafikkdata tabell. 
Ved bruk av en notebook er det viktig å huske begrensning på minne (~12GB i colab), slik at den veldig stor tabell kan sannsynligvis ikke lastes.
Her kan man benytte seg av en `LIMIT` clause som returnere bare X antall rader. 

In [ ]:
%%bigquery --project $project
SELECT
 *
FROM
  --todo: change to -> workshop.proddata
  `saga-trafikkdata-stm-39pt.spike_eu.import_prod_mars` 
LIMIT 1000

Kopier select statement inn til BigQuery UI og se på datamenge Query Validatoren angi. 
- Hvor mye data innegår i spørringen? 
- Er datamengden som prosesseres bakgrunnen avhengig avhengig av `LIMIT` clause?


BigQuery prises etter datamengenden som lagres eller analyseres. En spørring som analyserer mye data vil derfor koste mer.
Det finnes ulike måter å begrense datamengenden i en spørring. BigQuery behandler dataene kolonne-basert, dvs. jo flere kolonner man ta med i en spørring, desto mer kostnader oppstår.

Prøv å begrense spørringen ved a velge enkelte kolonner (f.eks. `trpId`)

In [ ]:
%%bigquery --project $project 
SELECT
  --<kolonne-navn>--
FROM
  `saga-trafikkdata-stm-39pt.spike_eu.import_prod_mars` 

- Hvor mye data innegår i spørringen? 
- Er datamengden avhengig av hvilke kolonne man spør mot?

I tillegg til standard data typer som f.eks FLOAT, INT og STRING har BigQuery også RECORDS (structs) og REPEATED kolonner (arrays). 

- RECORDS er sammensatt av én eller flere kolonner (et slags tabell inn i tabellen).  Det er mulig å ha en nesting av structs med flere nivåer. 
RECORDS typisk oppstå ved ingest av en nested JSON fil.
Kolonner i en RECORD kan refereres til med en `.` notasjon, f.eks. `total.volumeNumbers` 
- REPEATED kolonner inneholder arrays av en bestemt data type, f.eks. en array av FLOAT verdier. REPEATED kolloner kan brukes til å præ-aggregere data (én rad versus mange rader for å samle data).

Utforsk schema til trafikkdata tabellen. I BigQuery UI velg tabellen og trykk på `schema`. Hvilke 


### Aggregasjoner

Prøv å lage eg SUM() aggregasjon over total trafikkvolumen.

Hint: `volume` kolonne inneholder trafikkvolumen per måling og finnes under en nested RECORD:
  - total 
    - volumeNumbers 
      - volume

In [ ]:
%%bigquery --project $project 
SELECT
  --<kolonne-navn>--
FROM
  `saga-trafikkdata-stm-39pt.spike_eu.import_prod_mars`

Bruk SUM() aggregasjon sammen med en `group by` for å summere trafikkvolumen per trafikkregistreringspunkt `trpId` og sorter etter totalvolumen (fallende)


In [ ]:
%%bigquery --project $project 
SELECT
  --<kolonne-navn1>--,
  --<kolonne-navn2>--
FROM
  `saga-trafikkdata-stm-39pt.spike_eu.import_prod_mars`
GROUP BY
 --<kolonne-navn>--
ORDER BY
 --<kolonne-navn>--
DESC

#### Duplikat hantering

Datasettene i BigQuery er vanligvis denormalisert og det er vanlig å finne noen duplikator enten pga duplikasjon i underliggende datakilder eller som en resultat av avvik i ingestpipelinen. En enkel måte å finne disse duplikatene er ved å beregne total antall rader per en key som skal være unik, her f.eks. en kombinasjon av en tellepunkt ID og tidspunkt av måling. I spørringen under brukes det `HAVING` til filtrering av resultatene etter spørringen er kjørt.

In [ ]:
%%bigquery --project $project 
SELECT
  trpId, `from`, `to`, COUNT(*) antall_rader
FROM
  `spike_eu.import_prod_mars` 
GROUP BY 1,2,3
ORDER BY antall_rader 
DESC

Hvor mange rader har minst 1 duplikat?

Fjern duplikatene! 

Det finnes ulike muligheter å gjøre dette direkte i BigQuery. 
Hvis enkelte rader er **100%** lik er det mulig f.eks. å kjøre en enkelt `SELECT DISTINCT` spørring:

In [ ]:
%%bigquery --project $project 
WITH dummy AS (
  SELECT 
  "a" as id, 1 as value
  UNION ALL
  SELECT
  "a" as id, 1 as value --duplicate
  UNION ALL
  SELECT
  "b" as id, 2 as value
)
SELECT DISTINCT * FROM dummy

### Analytiske funksjoner

I vår tilfelle inneholder trafikkdata desverre rader der trafikkvolume har blitt oppdatert og vi har flere versjoner av den samme måling der vi vil bare ta vare på den siste versjonen i vår arbeidstabell.

I slike situasjoner er det f.eks. å bruke en [Analytiske funksjon](https://cloud.google.com/bigquery/docs/reference/standard-sql/analytic-function-concepts). Analytiske funksjoner kan brukes til å utføre aggregasjoner per gruppe lignende som group by, men resultatet returnerer **én rad per hver input rad**. 

Her kan vi bruke analytiske funksjon `ROW_NUMBER()` til å nummerere radene gruppert etter registereringspunkt (trpId), start (from)- og slutttidspunk  (to)og sortert fallende etter data importeringstidspunkt. Til slutt bruker spørringen en WHERE clause for å velge den første raden per gruppe.

In [ ]:
%%bigquery --project $project 
SELECT
  trpId,
  `from`,
  `to`,
  ROW_NUMBER() OVER(PARTITION BY trpId ORDER BY ingest_time DESC) AS rn
FROM
  `spike_eu.import_prod_mars`
WHERE 
  RN = 1

Spørsmål:
- Hva skjer når man fjerne WHERE clause?
- Hvordan ville man fått ut dataene til den *første* importeringstidspunkt (per hver gruppe)

### Joins
BigQuery støtter alle vanlige typer joins som left/right join, inner join, full outer join, cross join.

Lag en left outer join, som knytter sammen et utvalg av kolonner i trafikkdata med lokasjonsinformasjon om trafikkregistreringspunkter. Bruk `trpId` fra trafikkdata tabellen og `id` fra registreringspuntktabellen i JOIN statement.

In [ ]:
%%bigquery --project $project 
SELECT
  a.trpId,
  a.from,
  a.to,
  a.total.volumeNumbers.volume,
  b.location.coordinates.latLon
FROM
  `spike_eu.import_prod_mars` a
LEFT JOIN
  `spike_eu.trafikkregistreringspunkter` b
ON
  -- join statement -- 

### Views

Views er en enkel måte å lagre en spørringen der resultatene oppdateres med endringer i underliggende datakilder. 

Bruk JOIN-spørringen fra før som view definisjon under og lagre viewen i ditt  workshop prosjekt. Bruk følgende skjelettet:


In [ ]:
%%bigquery --project $project 
CREATE VIEW workshop.trafikkdata_view AS (
-- view definition -- 
)

Kjør en spørring mot viewen:

In [ ]:
%%bigquery --project $project 
SELECT 
 * 
FROM workshop.trafikkdata_view

Når man bruker en view som en del av analyser kan det være hensiktsmessig å lage en materialzed view. 

Materialized views lagrer delresultatene på en spørring som en fysisk tabell (cache) som automatisk oppdateres i regelmessige intervaller. 
En fordel med dette er at spørringer mot en materialized view er effektivere enn mot en vanlig view. 

En krav på materialiazed views er at de må inneholde et eller annet form av aggregasjon:

In [ ]:
%%bigquery --project $project 
CREATE MATERIALIZED VIEW workshop.traffikdata_view_materialized AS (
SELECT
  trpId, `from`, `to`, COUNT(*) antall_rader
FROM
  `spike_eu.import_prod_mars` 
group by 1,2,3

## Partisjonerte tabeller
BigQuery's forbruk av data (analysert datamengde) avhenger i stor grad av antall/størrelse av kolonner en spørring bruker. 

En `WHERE` clause hjelper generelt ikke med begrensningen av datamengden siden den tar effekt etter dataene er hentet.
Ett untak er spørring mot [partisjonerte tabeller](https://cloud.google.com/bigquery/docs/partitioned-tables), der en `WHERE` clause effektivt begrenser datamengden ved å velge partisjoner.

Vanligvis bruker vi tidspartisjonerte tabeller.
Tidspartisjonerte tabeller krever minst en kolonne som inneholder TIMESTAMP, DATETIME eller DATE verdier. I trafikkdata tabellen har vi flere slike kolonner
som angi både en tidspunkt av målingen (`from`, `to`) og en tidspunkt der dataene blir hentet inn i vårt system (`ingest_time`).



I cellen under oppretter vi en tidspartisjonert tabell.
- Først konverterer vi `from` kolonnen fra STRING til TIMESTAMP data type
- Så bruker vi en CREATE TABLE statement til å lage en ny tabell med tidspartisjonering på konverterte `from` kolonnen

In [10]:
%%bigquery --project $project 
/* 
Example how to parse timestamps from string
*/
WITH original AS (
  SELECT
    "2016-06-22T19:00+02:00[Europe/Oslo]" AS date_string,
)
SELECT
  date_string, 
  REGEXP_EXTRACT(date_string, r'\[([^\[\]]*)\]') timezone, 
  DATETIME(PARSE_TIMESTAMP("%Y-%m-%dT%H:%M%Ez %Z ", REGEXP_REPLACE(date_string, r'[\[\]]', ' '))) AS datetime_utc,
  DATETIME(PARSE_TIMESTAMP("%Y-%m-%dT%H:%M%Ez %Z ", REGEXP_REPLACE(date_string, r'[\[\]]', ' ')), REGEXP_EXTRACT(date_string, r'\[([^\[\]]*)\]') ) AS datetime_oslo
from original

,date_string,timezone,datetime_utc,datetime_oslo
0,2016-06-22T19:00+02:00[Europe/Oslo],Europe/Oslo,2016-06-22 17:00:00,2016-06-22 19:00:00


Lag en tabell som er partisjonert på en DATETIME/DATE kolonne. 

In [ ]:
%%bigquery --project $project 
CREATE TABLE spike_eu.trafikkdata_sub_partitioned  PARTITION BY DATE(datetime) 
AS (
SELECT
  a.trpId,
  -- <datetime transformation> datetime 
  a.from,
  a.to,
  a.total.volumeNumbers.volume,
  b.location.coordinates.latLon
FROM
  `spike_eu.import_prod_mars` a
LEFT JOIN
  `spike_eu.trafikkregistreringspunkter` b
ON
  a.trpId = b.Id
)

Sammenligne datamengden ved prosessering av følgende to spørringer:

In [ ]:
%%bigquery --project $project 
SELECT 
* 
FROM spike_eu.trafikkdata_sub_partitioned` 
WHERE DATE(datetime) = "2019-11-01"

In [ ]:
%%bigquery --project $project 
SELECT 
* 
FROM spike_eu.trafikkdata_sub_partitioned` 

## GIS funksjoner
Støtte for noen GIS-funksjoner gjør BigQuery også til en kraftig analyse verktøy innen geoprosessering. BigQuery bruker en egen datatype `GEOGRAPHY` data type til å hantere objekter med geolokasjoner og tilbyr ulike parser som f.eks. `ST_GEOGFROMTEXT()` som hjelper med å transformere en WKT string til en geografisk object.

En oversikt over støttede GIS-funksjoner finnes her: 

https://cloud.google.com/bigquery/docs/reference/standard-sql/geography_functions 

En geografisk punkt kan lages med ST_GEOGPOINT() funksjon ved bruk av GPS lengdregrad og breddegrad. ST_GEOGPOINT() krever at lengdegrad og breddegrad er i FLOAT64 data type. Det er mulig å bruke `CAST(<string_kolonne> AS FLOAT64) <ny_kolonnenavn>`

In [ ]:
%%bigquery --project $project
SELECT 
  ST_GEOGPOINT(--<velg lengdegrad kolonne> , <velg breddegrad kolonne>-- ) 
FROM 
  `spike_eu.trafikkregistreringspunkter`

Kjør en spørring som henter ut registreringspunkter i et gitt området.

In [ ]:
%%bigquery --project $project
SELECT 
 ST_GEOGPOINT() 
FROM 

## Brukerfunksjoner (UDFs)
Av og til mangles det en enkel funksjon som er ikke definert i standard SQL eller i BigQuery's ekstra funksjoner. Det er enkelt å definere og bruke egne funksjoner (UDFs) enten temporær i en skript eller lagre dem globalt i et prosjekt. 

In [ ]:
-- Eksempel funksjonsdefinisjon

Det er også mulig å bruke UDFs til å integrer ekstern javascript kode, f.eks. ved å kalle en java script bibliotek fil som ligger i en GCS bøtte. 
Her brukes det en UDF som interface/wrapper:

Definer en buffer funksjon

Bruk buffer funksjon sammen med intersect spørring.

In [ ]:
-- ST_BUFFER eksempel

## (BigQuery ML)

Demo av trafikkvolumen forecasting med enkel regresjon. 